# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## Setup

In [ ]:
# import libraries
import logging

import numpy as np
from scipy import optimize

import utils

In [ ]:
# set constants
yaw_scale = 30

## Baseline values

In [ ]:
# run baseline simulation
sim_res_base = utils.run_sim()

In [ ]:
# ensure probabilities (wind direction and speed) total 1
Sector_frequency = sim_res_base.Sector_frequency
if not np.isclose(Sector_frequency.sum(), 1):
    logging.warning(
        f"Sector frequency renormalised as total probability was {Sector_frequency.sum().values}"
    )
    Sector_frequency = Sector_frequency / Sector_frequency.sum()
P = sim_res_base.P
if not np.isclose(P.sum(), 1):
    logging.warning(f"P renormalised as total probability was {P.sum().values}")
    P = P / P.sum()

In [ ]:
# calculate baseline metrics
aep_base, lcoe_base, _ = utils.calc_metrics(
    sim_res=sim_res_base,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)
lcoe_direction_base, _, lcoe_overall_base, _ = utils.aggregate_metrics(
    aep=aep_base, lcoe=lcoe_base, Sector_frequency=Sector_frequency
)

## Optimise across for a single wind direction

In [ ]:
# define objective function
yaw_shape = (len(sim_res_base.wt), len(sim_res_base.wd), len(sim_res_base.ws))


def obj_single(yaw_norm):
    sim_res = utils.run_sim(yaw=yaw_norm.reshape(yaw_shape) * yaw_scale)
    aep, lcoe, _ = utils.calc_metrics(
        sim_res=sim_res,
        sim_res_base=sim_res_base,
        Sector_frequency=Sector_frequency,
        P=P,
    )
    lcoe_direction, _, lcoe_overall, _ = utils.aggregate_metrics(
        aep=aep, lcoe=lcoe, Sector_frequency=Sector_frequency
    )
    # display(lcoe_direction)
    # display(lcoe_direction_base)
    # display(lcoe_overall)
    # display(lcoe_overall_base)
    obj = (lcoe_overall / lcoe_overall_base).values.tolist()
    return obj

In [ ]:
assert obj_single(np.zeros(yaw_shape)) == 1

In [ ]:
# obj_single(np.arange(np.prod(yaw_shape)).reshape(yaw_shape)/1000/yaw_scale)

In [ ]:
# n = 0

# def _callback(x):
#     global n
#     print(f"--- Iteration {n} ---")
#     print(x)
#     n += 1

# res = optimize.minimize(
#     fun=obj_single,
#     x0=np.zeros_like(sim_res_base.Power).ravel(),
#     callback=_callback
# )
# res

In [ ]:
# np.round(res.x * yaw_scale, 1)